<a href="https://colab.research.google.com/github/joyfulspace/ADP/blob/master/07_%EA%B8%B0%EA%B3%84%ED%95%99%EC%8A%B5_8_%EC%97%B0%EA%B4%80%EA%B7%9C%EC%B9%99%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 연관규칙분석
- 주어진 어떤 집합에서 원소들 간에 존재하는 규칙을 찾는 것
  (변수 간의 흥미로운 관계를 발견하기 위한 규칙기반 머신러닝)
- POS(Point Of Sale) 시스템에서 제품 간의 규칙성을 발견하는 기법
- 장바구니 분석, 생물 정보학, 웹 사용 마이닝 등에 활용
- 집합의 가능한 부분집합을 모두 구해야 해서 원소의 수가 많으면 연산량이 많아짐
  - 빈발 항목 집합: 부분 집합 중에서 빈번하게 나오는 아이템들의 집합
    - 예시: Apriori(선험 규칙)
  - 비빈발 항목 집합: 부분 집합 중에서 빈번하게 나오지 않는 아이템들의 집합
  
- 개념
  - S(X->Y): support(지지도). $n(X\cup Y)/N=P(X\cup Y)$
    - 전체 건수 중에서 X와 Y가 모두 포함되어 있는 건수의 비
    - 전체 집합에서 어느 정도 점유율을 차지하는지 표현
  - c(X->Y): confidence(신뢰도). $n(X\cup Y)/n(X)=P(Y|X)$
    - 항목 X를 포함하는 건수 중 X와 Y를 모두 포함하는 건수의 비
    - 어떤 규칙이 어느 정도 신뢰성이 있는지 표현
  - 향상도(Lift): $\frac{신뢰도}{P(Y)}$
    - $Lift(A,B)=\frac{c(A->B)}{s(B)}=P(Y|X)/P(Y)=P(X\cup Y)/P(X)P(Y)$
    - 그냥 B를 구매한 경우와 A를 구매한 후에 B를 구매한 경우의 차이가 클수록 큰 의미를 가짐
    - 상품 Y를 구매한 경우보다 얼마나 향상이 되었는지 보는 것
    - 1보다 크면 양의 상관, 1보다 작으면 음의 상관
    - 절대값이 1보다 클수록 좋음
  
- 규칙을 만들 때
  1. 최소 지지도를 먼저 결정 (빅데이터의 경우 지지도가 작을 수 있음)
  2. 최소 지지도보다 높은 지지도 중에서 신뢰도가 높은 규칙을 찾음
  3. 신뢰도를 높이면서 더 좋은 규칙을 찾아냄
  
- 선험 규칙: 모든 경우의 규칙패턴을 관찰하기 위해 필요한 연산 규칙
   1. 한 항목 집합이 빈발하면 그 집합의 모든 부분 집합은 빈발 항목 집합이다.
   2. 한 항목 집합이 비빈발이면 그 항목 집합을 포함하는 모든 집합은 비빈발 항목 집합이다.
   - 연산 비용을 절약 가능
   - 너무 자명해서 수학적인 증명이 필요하지 않음

## 실습1

In [1]:
!pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=0ac4ad75e4929837759277e1ed51066a6bbacd246ad66fe400ad2432a5ad2df6
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori

In [8]:
!git clone https://github.com/joyfulspace/ADP.git

Cloning into 'ADP'...
remote: Enumerating objects: 2526, done.
remote: Counting objects: 100% (2219/2219), done.
remote: Compressing objects: 100% (2213/2213), done.
remote: Total 2526 (delta 44), reused 2135 (delta 4), pack-reused 307
Receiving objects: 100% (2526/2526), 56.05 MiB | 25.92 MiB/s, done.
Resolving deltas: 100% (166/166), done.


In [9]:
store_data = pd.read_csv('ADP/data/store_data.csv', header=None)

In [12]:
store_data.shape

(7501, 20)

In [10]:
store_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
records = []
for i in range(0, store_data.shape[0]):
  records.append([str(store_data.values[i,j]) for j in range(0, store_data.shape[1])])

In [14]:
association_rules = apriori(records, min_support=0.0045, min_confidence=0.2, min_lift=3, min_length=2) 
                                      # 최저값에서 조금씩 높여가며 규칙 탐색.            빈발항목집합의 개수
association_results = list(association_rules)

In [15]:
len(association_results)

48

In [16]:
for item in association_results:
  pair = item[0]
  items = [x for x in pair]
  print('Rule: '+items[0]+' -> '+items[1])

  # 지지도
  print('Support: '+str(item[1]))

  # 신뢰도 & 리프트
  print('Confidence: '+str(item[2][0][2]))
  print('Lift: '+str(item[2][0][3]))
  print('===================================')

Rule: chicken -> light cream
Support: 0.004532728969470737
Confidence: 0.29059829059829057
Lift: 4.84395061728395
Rule: mushroom cream sauce -> escalope
Support: 0.005732568990801226
Confidence: 0.3006993006993007
Lift: 3.790832696715049
Rule: pasta -> escalope
Support: 0.005865884548726837
Confidence: 0.3728813559322034
Lift: 4.700811850163794
Rule: herb & pepper -> ground beef
Support: 0.015997866951073192
Confidence: 0.3234501347708895
Lift: 3.2919938411349285
Rule: tomato sauce -> ground beef
Support: 0.005332622317024397
Confidence: 0.3773584905660377
Lift: 3.840659481324083
Rule: whole wheat pasta -> olive oil
Support: 0.007998933475536596
Confidence: 0.2714932126696833
Lift: 4.122410097642296
Rule: shrimp -> pasta
Support: 0.005065991201173177
Confidence: 0.3220338983050847
Lift: 4.506672147735896
Rule: chicken -> nan
Support: 0.004532728969470737
Confidence: 0.29059829059829057
Lift: 4.84395061728395
Rule: frozen vegetables -> chocolate
Support: 0.005332622317024397
Confidence:

## 실습2

In [5]:
# 엄마의 장바구니 입력
dataset = [['우유', '양파', '땅콩', '호두', '계란', '요거트'],
           ['감자', '양파', '두유', '호두', '계란', '요거트'],
           ['우유', '사과', '호두', '계란'],
           ['우유', '새우깡', '옥수수', '호두', '요거트'],
           ['옥수수', '감자', '양파', '호두', '아이스크림', '계란']]

In [17]:
# 트랜젝션 데이터로 바꿈. 장바구니 데이터를 숫자로 변경
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [19]:
df.head()

,감자,계란,두유,땅콩,사과,새우깡,아이스크림,양파,옥수수,요거트,우유,호두
0,False,True,False,True,False,False,False,True,False,True,True,True
1,True,True,True,False,False,False,False,True,False,True,False,True
2,False,True,False,False,True,False,False,False,False,False,True,True
3,False,False,False,False,False,True,False,False,True,True,True,True
4,True,True,False,False,False,False,True,True,True,False,False,True


In [20]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# 지지도가 0.6이상인 규칙을 추출한다.
apriori(df, min_support=0.6)

,support,itemsets
0,0.8,(1)
1,0.6,(7)
2,0.6,(9)
3,0.6,(10)
4,1.0,(11)
5,0.6,"(1, 7)"
6,0.8,"(1, 11)"
7,0.6,"(11, 7)"
8,0.6,"(9, 11)"
9,0.6,"(10, 11)"


In [21]:
apriori(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,0.8,(계란)
1,0.6,(양파)
2,0.6,(요거트)
3,0.6,(우유)
4,1.0,(호두)
5,0.6,"(양파, 계란)"
6,0.8,"(계란, 호두)"
7,0.6,"(양파, 호두)"
8,0.6,"(호두, 요거트)"
9,0.6,"(우유, 호두)"


In [22]:
# 아이템셋의 길이를 구하여 컬럼에 추가한다.
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.8,(계란),1
1,0.6,(양파),1
2,0.6,(요거트),1
3,0.6,(우유),1
4,1.0,(호두),1
5,0.6,"(양파, 계란)",2
6,0.8,"(계란, 호두)",2
7,0.6,"(양파, 호두)",2
8,0.6,"(호두, 요거트)",2
9,0.6,"(우유, 호두)",2


In [23]:
frequent_itemsets[ (frequent_itemsets['length']==2)&(frequent_itemsets['support']>=0.8) ]

,support,itemsets,length
6,0.8,"(계란, 호두)",2


In [24]:
frequent_itemsets[frequent_itemsets['itemsets']=={'양파', '계란'}]

,support,itemsets,length
5,0.6,"(양파, 계란)",2


In [25]:
# 연관규칙 생성
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(양파),(계란),0.6,0.8,0.6,1.00,1.25,0.12,inf
1,(계란),(양파),0.8,0.6,0.6,0.75,1.25,0.12,1.6
2,(계란),(호두),0.8,1.0,0.8,1.00,1.00,0.00,inf
3,(호두),(계란),1.0,0.8,0.8,0.80,1.00,0.00,1.0
4,(양파),(호두),0.6,1.0,0.6,1.00,1.00,0.00,inf
5,(호두),(양파),1.0,0.6,0.6,0.60,1.00,0.00,1.0
6,(호두),(요거트),1.0,0.6,0.6,0.60,1.00,0.00,1.0
7,(요거트),(호두),0.6,1.0,0.6,1.00,1.00,0.00,inf
8,(우유),(호두),0.6,1.0,0.6,1.00,1.00,0.00,inf
9,(호두),(우유),1.0,0.6,0.6,0.60,1.00,0.00,1.0


In [26]:
rules[(rules['lift']>1)&(rules['confidence']>=0.6)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(양파),(계란),0.6,0.8,0.6,1.00,1.25,0.12,inf
1,(계란),(양파),0.8,0.6,0.6,0.75,1.25,0.12,1.6
11,"(양파, 호두)",(계란),0.6,0.8,0.6,1.00,1.25,0.12,inf
12,"(계란, 호두)",(양파),0.8,0.6,0.6,0.75,1.25,0.12,1.6
13,(양파),"(계란, 호두)",0.6,0.8,0.6,1.00,1.25,0.12,inf
14,(계란),"(양파, 호두)",0.8,0.6,0.6,0.75,1.25,0.12,1.6
